# Object Tracking

In [8]:
%pip install opencv-python numpy tqdm matplotlib

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.6 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.6 kB ? eta -:--:--
     ------------- ------------------------- 20.5/57.6 kB 72.6 kB/s eta 0:00:01
     ------------- ------------------------- 20.5/57.6 kB 72.6 kB/s eta 0:00:01
     --------------------------- ---------- 41.0/57.6 kB 140.3 kB/s eta 0:00:01
     --------------------------- ---------- 41.0/57.6 kB 140.3 kB/s eta 0:00:01
     --------------------------- ---------- 41.0/57.6 kB 140.3 kB/s eta 0:00:01
     --------------------------- ---------- 41.0/57.6 kB 140.3 kB/s eta 0:00:01
     -------------------------


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import numpy as np
import math
import glob
import os
import copy
from tqdm import tqdm
from matplotlib import pyplot as plt

# Helper Functions

In [3]:
def get_rectangle(frame):
    # Select ROI on certain image by marking it with a rectangle
    # Returns rectangle coordinates as [x,y,w,h]

    # Define a variable to store the coordinates of the rectangle
    rect = [0, 0, 0, 0]

    # Make a copy of the frame
    img = frame.copy()

    # Define the mouse callback function
    def select_rectangle(event, x, y, flags, param):
        nonlocal rect
        if event == cv2.EVENT_LBUTTONDOWN:
            rect[0] = x
            rect[1] = y
        elif event == cv2.EVENT_LBUTTONUP:
            rect[2] = x - rect[0]
            rect[3] = y - rect[1]

    # Set the mouse callback function for the window
    cv2.namedWindow("Select rectangle")
    cv2.setMouseCallback("Select rectangle", select_rectangle)
    # draw rectangle


    while(True):
        # Show the frame
        cv2.rectangle(img, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 2)
        cv2.imshow("Select rectangle", img)

        # Exit if the user presses the 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the display window
    cv2.destroyAllWindows()

    return rect

In [4]:
def getWarpMat(p):
    # Returns (2x3) affine transformation matrix given 6 transform parameters

    W = np.array([[1 + p[0],     p[2],  p[4]],
                  [    p[1], 1 + p[3],  p[5]]])
    return W

In [5]:
def getUnrolledPts(Image, ROI):
    # Flattens 2D region of image into 1D array based on ROI provided

    x, y, w, h = ROI
    # Array to store flattened pixel locations, each element consists of [x, y, 1]
    unrolledPts = np.empty([h*w,3])

    n = 0
    for i in range(x, x+w):
        for j in range(y, y+h):
            unrolledPts[n,0] = i
            unrolledPts[n,1] = j
            unrolledPts[n,2] = 1
            n+=1

    # Array to store flattened pixel values, each element consists of intensity value from original image
    unrolledIntensity = np.empty(h*w)
    for i, pt in enumerate(unrolledPts):
        unrolledIntensity[i] = Image[int(pt[1]),int(pt[0])]

    return unrolledPts, unrolledIntensity


def warpPoints(Image, warpMat, templatePts, gradientX, gradientY):
    # Performs affine warp on a given set of points using a given warp matrix
    # Returns warped pixel locations, warped pixel values, and warped gradient values

    h, w = Image.shape

    # Warping points
    warpedImgPts = np.matmul(warpMat, templatePts.T).astype(int)

    # Clipping points out of image dimensions
    warpedImgPts[1,:] = np.clip(warpedImgPts[1,:], 0, h-1)
    warpedImgPts[0,:] = np.clip(warpedImgPts[0,:], 0, w-1)

    # Assigning Intensities and gradients to warped points
    warpedImgIntensities = Image[warpedImgPts[1,:].astype(int),warpedImgPts[0,:].astype(int)]
    gradXValues = gradientX[warpedImgPts[1,:].astype(int),warpedImgPts[0,:].astype(int)]
    gradYValues = gradientY[warpedImgPts[1,:].astype(int),warpedImgPts[0,:].astype(int)]

    return warpedImgPts, warpedImgIntensities, gradXValues, gradYValues


def getJacobian(templatePts):
    # Returns jacobian matrix based on given set of [x,y] corrdinates

    x = np.array(templatePts[...,0])
    y = np.array(templatePts[...,1])

    ones = np.ones(templatePts.shape[0])
    zeros = np.zeros(templatePts.shape[0])

    row1 = np.stack((x, zeros, y, zeros, ones, zeros), axis=1)
    row2 = np.stack((zeros, x, zeros, y, zeros, ones), axis=1)
    jacob = np.stack((row1, row2), axis=1)

    return jacob

# Functions for saving data

In [6]:
def save_as_npy(frames, filename):

    if os.path.exists(filename):
        print("File already exists")
        return
    np.save(filename, frames)
    print("Saved as", filename)

def save_as_gif(frames, filename, fps=10, duration=0.02):

    if os.path.exists(filename):
        print("File already exists")
        return
    frames = frames[::2]
    # reduce the resolution of the frames
    frames = [cv2.resize(frame, (0,0), fx=0.5, fy=0.5) for frame in frames]
    imageio.mimsave(filename, frames, fps=fps, duration=duration, subrectangles=True)
    print("Saved as", filename)

# Lucas Kanade Tracker Implementation

In [18]:
def getDeltaP(b, gradImage, templatePts):

    jacobian = getJacobian(templatePts)
    gradImage = np.expand_dims(gradImage, axis=1)

    A = np.matmul(gradImage, jacobian)
    A = np.squeeze(A, axis=1)

    Hessian = np.matmul(A.T, A)
    AT_b = np.matmul(A.T, b)

    deltaP = np.matmul(np.linalg.pinv(Hessian), AT_b)

    return deltaP

In [17]:
def lucasKanadeTracker(templatePts, templateIntensity, Image, p):

    threshold = 0.035
    maxIter = 150
    deltaP_norm = np.inf

    gradientX = cv2.Sobel(Image,cv2.CV_64F,1,0,ksize=3)
    gradientY = cv2.Sobel(Image,cv2.CV_64F,0,1,ksize=3)

    for i in range(maxIter):

        W = getWarpMat(p)
        warpedImgPts, warpedImgIntensity, gradXValues, gradYValues = warpPoints(Image, W, templatePts, gradientX, gradientY)

        error = templateIntensity - warpedImgIntensity

        gradImage = np.stack((gradXValues, gradYValues), axis=1)

        deltaP = getDeltaP(error, gradImage, templatePts)
        deltaP_norm = np.linalg.norm(deltaP)

        p += deltaP

        if (deltaP_norm < threshold):
            break

    return W,p

In [16]:
def trackObject(frames, ROI):

  tracker_output = []
  roi_Pt1, roi_Pt2 = np.array([ROI[0], ROI[1], 1]), np.array([ROI[0]+ROI[2], ROI[1]+ROI[3], 1])

  template = frames[...,0]
  templatePts, templateIntensity = getUnrolledPts(template, ROI)
  mean_template = np.mean(template)

  p = np.zeros(6, dtype = float)

  for i in tqdm(range(1, frames.shape[-1])):

    frame_current = copy.deepcopy(frames[...,i])
    frame_display = frame_current.copy()

    # Normalizing the intensity for new image with respect to template image
    mean_frame = np.mean(frame_current)
    frame_current = frame_current.astype(float)
    frame_current = frame_current *(mean_template/mean_frame)

    # Lucas Kanade Tracker
    warpMat, p = lucasKanadeTracker(templatePts, templateIntensity, frame_current, p)

    roi_Pt1_new = np.matmul(warpMat, roi_Pt1).astype(int)
    roi_Pt2_new = np.matmul(warpMat, roi_Pt2).astype(int)

    cv2.rectangle(frame_display, tuple(roi_Pt1_new), tuple(roi_Pt2_new), (0, 0, 255), 2)
    tracker_output.append(frame_display)

  return tracker_output

# Loading Data

In [8]:
loaded_array = np.load('./Data/landing.npy')
print(f"{loaded_array.shape}")

(1000, 1100, 50)


In [9]:
data_dir = './Data/'
videos = {}

for vid in os.listdir(data_dir):
        if vid.endswith('.npy'):
            loaded_array = np.load(data_dir + vid)
            videos[vid.split('.')[0]] = loaded_array

print('Videos Loaded: ', list(videos.keys()))

Videos Loaded:  ['car2', 'landing', 'tracker_car2', 'tracker_landing']


In [11]:
def play_video(frames, type='array'):

    frame_count = lambda : frames.shape[-1] if type == 'array' else len(frames)
    frame_idx = lambda i: frames[...,i] if type == 'array' else frames[i]

    print("Playing video ....")
    print("Press q to exit")
    for i in range(frame_count()):
        cv2.imshow('frame', frame_idx(i))
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

In [12]:
play_video(videos['car2'], type='array')

Playing video ....
Press q to exit


# Applying Tracker

## Car_2

In [13]:
selection = 'car2'
frames = videos[selection]

In [14]:
ROI = cv2.selectROI(frames[...,0])
cv2.destroyAllWindows()
print('Selected Region (x,y,w,h): ', ROI)

Selected Region (x,y,w,h):  (52, 110, 104, 51)


In [19]:
tracker_car2 = trackObject(frames, ROI)

  0%|          | 0/414 [00:00<?, ?it/s]

100%|██████████| 414/414 [00:21<00:00, 19.04it/s]


In [26]:
%pip install imageio

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for imageio from https://files.pythonhosted.org/packages/c0/69/3aaa69cb0748e33e644fda114c9abd3186ce369edd4fca11107e9f39c6a7/imageio-2.33.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/313.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/313.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/313.3 kB ? eta -:--:--
   --- ----------------------------------- 30.7/313.3 kB 330.3 kB/s eta 0:00:01
   ------- ------------------------------- 61.4/313.3 kB 469.7 kB/s eta 0:00:01
   -------------- ----------------------- 122.9/313.3 kB 654.9 kB/s eta 0:00:01
   --------------------------- ---------- 225.3/313.3 kB 981.9 kB/s eta 0:00:01
   ---------------------------------------  307.2/313.3 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 313.3/313.3 kB 1.1 MB/s eta 0:00:00
Note: you may need to restar


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
import imageio
# save numpy array as npy file
%pip install imageio
save_path = f'{data_dir}tracker_{selection}.npy'
save_as_npy(tracker_car2, save_path)
save_as_gif(tracker_car2, f'{data_dir}tracker_{selection}.gif')

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Saved as ./Data/tracker_car2.npy



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Saved as ./Data/tracker_car2.gif


## Landing

In [33]:
selection = 'landing'
frames = videos[selection]

In [34]:
ROI = cv2.selectROI(frames[...,0])
cv2.destroyAllWindows()
print('Selected Region (x,y,w,h): ', ROI)

Selected Region (x,y,w,h):  (425, 84, 142, 50)


In [35]:
tracker_landing = trackObject(frames, ROI)

100%|██████████| 49/49 [00:06<00:00,  7.86it/s]


In [36]:
# save numpy array as npy file
save_path = f'{data_dir}tracker_{selection}.npy'
save_as_npy(tracker_landing, save_path)
save_as_gif(tracker_landing, f'{data_dir}tracker_{selection}.gif')

Saved as ./Data/tracker_landing.npy
Saved as ./Data/tracker_landing.gif
